

---


# **Twitter Data Collection**
#### ***Look before you Leap : Leveraging Predictive Models to Improve Automotive Safety and Travel Time***



---



**Final Project**
CSCI 35300/79502   |   Fall 2020 

##### Hannah Do








#### Used Snscrape and Tweepy to collect queried tweets :

In [1]:
! pip3 install snscrape

In [2]:
! pip install tweepy

In [103]:
# Storing twitter URLs in according folders

## TotalTrafficNYC data inconsistent  - sparse at the beginning of the year
## NYPD traffic and NY_DOT - too much irrelevant information / news or ad centered tweets

! snscrape twitter-search "@511NY since:2020-06-01 until:2020-06-15" > data_twitter/511_6a.txt
! snscrape twitter-search "@511NY since:2020-06-16 until:2020-07-01" > data_twitter/511_6b.txt

! snscrape twitter-search "@511NY since:2020-07-02 until:2020-07-15" > data_twitter/511_7a.txt
! snscrape twitter-search "@511NY since:2020-07-16 until:2020-08-01" > data_twitter/511_7b.txt

! snscrape twitter-search "@511NY since:2020-08-02 until:2020-08-15" > data_twitter/511_8a.txt
! snscrape twitter-search "@511NY since:2020-08-16 until:2020-09-01" > data_twitter/511_8b.txt

! snscrape twitter-search "@511NY since:2020-09-02 until:2020-09-15" > data_twitter/511_9a.txt
! snscrape twitter-search "@511NY since:2020-09-16 until:2020-10-01" > data_twitter/511_9b.txt

! snscrape twitter-search "@511NY since:2020-10-02 until:2020-10-15" > data_twitter/511_10a.txt
! snscrape twitter-search "@511NY since:2020-10-16 until:2020-11-01" > data_twitter/511_10b.txt

! snscrape twitter-search "@511NY since:2020-11-02 until:2020-11-15" > data_twitter/511_11a.txt
! snscrape twitter-search "@511NY since:2020-11-16 until:2020-12-01" > data_twitter/511_11b.txt



In [1]:

# Import all the libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

import datetime, string, re
import sys

import tweepy
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from collections import Counter


sid = SentimentIntensityAnalyzer()
# initializing Vader Sentiment Analyzer


print('All Libraries Sucessfully Imported')


All Libraries Sucessfully Imported


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/doguma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/doguma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


---
#### Twitter Developer Accounts required for API keys :

In [6]:

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [106]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at}
#             tweet_elem = {
#                      "tweet":status.full_text,
#                 "date":status.created_at
#                         }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    return empty_data

- Timeline segmented to smaller units due to long processing hours :

In [107]:

# '511_6a', '511_6b', '511_7a', '511_7b', '511_8a', '511_8b', '511_9a', '511_9b', '511_10a', '511_10b', '511_11a'
t_files = ['511_6a']


-  txt files (link of the tweets) converted to csv files (content of the tweets) :

In [109]:

for t in t_files:

    tweet_url = pd.read_csv("data_twitter/" + t +".txt", index_col= None, header = None, names = ["links"])
    tweet_url.head()

    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)

    ids = tweet_url['id'].tolist()

    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1


    for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        temp2 = pd.DataFrame(fetch_tw(batch))
        temp2.to_csv("data_twitter/" + t +".csv", mode="a")


- each csv file (content of the tweets) read as dataframe :

- ***start from here for geocoding process***

In [2]:
# 511_6a.csv, 511_6b.csv, 511_7a.csv, 511_7b.csv,
# 511_8a.csv, 511_8b.csv, 511_9a.csv, 511_9b.csv,
# 511_10a.csv, 511_10b.csv, 511_11a.csv, 511_11b.csv

# processed without a loop since each file takes a long time

tweet_text = pd.read_csv("data_twitter/511_6a.csv", index_col= None, header = None)
tweet_text

temp = pd.DataFrame()
temp['tweet'] = tweet_text[3]

temp.drop(temp.head(1).index,inplace=True)

temp['score_neutral'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[1])
# get neutral sentiment of the tweet

temp['score_polar'] = temp['tweet'].apply(lambda x: list(sid.polarity_scores(str(x)).values())[3])
# get compound score - overall polar sentiment of the tweet

temp


,tweet,score_neutral,score_polar
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000
...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000


- text processing

In [3]:
# separate camel cases and remove punctuations
# not needed for geocoding

def separate_camelcase(text):
    text = re.sub(r'\'s', '',text)
    text = re.sub(r'\'', '',text)
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub(r"([a-z])([A-Z]+)", r"\1 \2", str(text))

    return text


# Replace acronyms and remove redundant words

def replace_acronyms(text):
    # text  = "".join([char for char in text if char not in excluded_words])  
    text = re.sub('i76', ' interstate 76 ', text)
    text = re.sub('i84', ' interstate 84 ', text)
    text = re.sub('i87', ' interstate 87 ', text)
    text = re.sub('i91', ' interstate 91 ', text)
    text = re.sub('i95', ' interstate 95 ', text)
    text = re.sub('i190', ' interstate 190 ', text)
    text = re.sub('i295', ' interstate 295 ', text)
    text = re.sub('i878', ' interstate 878 ', text)
    text = re.sub('us 9', ' U.S. route 9 ', text)
    text = re.sub('us9', ' U.S. route 9 ', text)
    text = re.sub('us 130', ' U.S. route 130 ', text)
    text = re.sub('us130', ' U.S. route 130 ', text)
    
    text = re.sub('eb', ' east bound ', text)
    text = re.sub('sb', ' south bound ', text)
    text = re.sub('nb', ' north bound ', text)
    text = re.sub('wb', ' west bound ', text)

    text = re.sub('queens', ' queens ', text)
    text = re.sub('nj', ' new jersey ', text)
    text = re.sub('nyc', ' new york city ', text)
    text = re.sub('brooklyn', ' brooklyn ', text)
    text = re.sub('new york', ' new york ', text)        
    text = re.sub('staten island', ' staten island ', text)
    text = re.sub('newyork', ' new york ', text)
    text = re.sub('nypd', '', text)
    text = re.sub('ny', ' new york ', text)
    text = re.sub('bronx', ' bronx ', text)
#     text = re.sub('accident', '', text)
#     text = re.sub('parks', '', text)
#     text = re.sub('sirens', '', text)
#     text = re.sub('incident', '', text)
    text = re.sub('rte', ' route ', text)
    text = re.sub('rt', ' route ', text)
    text = re.sub('bklyn', ' brooklyn ', text)
    text = re.sub('bk', ' brooklyn ', text)
    text = re.sub('way', ' way ', text)
    text = re.sub('expy', ' expressway ', text)
#     text = re.sub('crash', '', text)
    text = re.sub('aka', '', text)
    text = re.sub('bqe', ' brooklyn queens expressway ', text)
    text = re.sub('511ny', '', text)

    return text 

temp['separate camel cases'] = temp['tweet'].apply(lambda x: separate_camelcase(x))

temp['replace acronyms'] = temp['separate camel cases'].apply(lambda x: x.lower())

temp['replace acronyms'] = temp['replace acronyms'].apply(lambda x: replace_acronyms(x))

temp

,tweet,score_neutral,score_polar,separate camel cases,replace acronyms
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000,Updated Incident on River Line Both directio...,updated incident on river line both directio...
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...
...,...,...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,Construction on NJ3 Both directions from Park...,construction on new jersey 3 both directions...
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,Cleared Incident on M14DBus at 14th Street,cleared incident on m14dbus at 14th street
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960,Construction on NY9A Both directions from Low...,construction on new york 9a both directions ...
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,Construction on Q12Bus at Alameda Ave https ...,construction on q12bus at alameda ave https ...


In [4]:
stopword = nltk.corpus.stopwords.words('english')

In [5]:

# Tokenize the text
def tokenization(text):
    text = re.split('\W+', text)
    return text

temp['remove stopwords'] = temp['replace acronyms'].apply(lambda x: tokenization(x))


# Remove stopwords

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: remove_stopwords(x))

temp.head(10)

# Lemmatization and Stemming skipped due to losing of necessary word forms

# Convert the tokens back to string

def listToString(s):  
    str1 = ""  
      
    for ele in s:  
        str1 += ele + " "  
      
    return str1

temp['remove stopwords'] = temp['remove stopwords'].apply(lambda x: listToString(x))
temp


,tweet,score_neutral,score_polar,separate camel cases,replace acronyms,remove stopwords
1,Cleared: Incident on #SouthernStateParkway WB ...,0.865,0.1027,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...,cleared incident southern state park way west ...
2,Updated: Incident on #RiverLine Both direction...,1.000,0.0000,Updated Incident on River Line Both directio...,updated incident on river line both directio...,updated incident river line directions trenton...
3,Cleared: Incident on #NJ139Lower EB at New Jer...,0.877,0.1027,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...,cleared incident new jersey 139lower east boun...
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,1.000,0.0000,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...,construction interstate 84 e east bound exit 3...
5,Incident on #NJ181 Both directions at CR 517/S...,1.000,0.0000,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...,incident new jersey 181 directions cr 517 spa ...
...,...,...,...,...,...,...
27083,Construction on #NJ3 Both directions from Park...,1.000,0.0000,Construction on NJ3 Both directions from Park...,construction on new jersey 3 both directions...,construction new jersey 3 directions park ave ...
27084,Cleared: Incident on #M14DBus at 14th Street.,0.811,0.1027,Cleared Incident on M14DBus at 14th Street,cleared incident on m14dbus at 14th street,cleared incident m14dbus 14th street
27085,Construction on #NY9A Both directions from Low...,0.855,-0.2960,Construction on NY9A Both directions from Low...,construction on new york 9a both directions ...,construction new york 9a directions lower stre...
27086,Construction on #Q12Bus at Alameda Ave. https:...,1.000,0.0000,Construction on Q12Bus at Alameda Ave https ...,construction on q12bus at alameda ave https ...,construction q12bus alameda ave https co dov n...


#### Geocoding on each text through ArcGIS : returns location (longitude, latitude) of the text
- takes at least 5 hours to process

In [33]:

from arcgis.gis import GIS
from arcgis.geocoding import geocode
my_gis = GIS()



In [44]:

temp2 = pd.DataFrame()
temp2 = temp.iloc[:25]

temp2["longitude"] = 0.000
temp2["latitude"] = 0.000



<ipython-input-44-0627a92c5e28>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2["longitude"] = 0.000
<ipython-input-44-0627a92c5e28>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2["latitude"] = 0.000


In [45]:
# adding geocode on each string, if no geocode retrieved - return (0,0)

i=1

while i <len(temp2['remove stopwords'])+1:
    string_temp = str(temp2['remove stopwords'][i])
    string_temp = string_temp + ' new york'
    
    if len(string_temp) < 100:    # maximum string length limit for arcgis geocoding
        locat = geocode(string_temp)
        if (locat):
            temp2['longitude'][i] = locat[0]['location']['x']
            temp2['latitude'][i] = locat[0]['location']['y']
        else:
            temp2['latitude'][i] = 0
            temp2['longitude'][i] = 0            
    else:
        temp2['latitude'][i] = 0
        temp2['longitude'][i] = 0
    i += 1
        
        


<ipython-input-45-97308dacfe21>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['longitude'][i] = locat[0]['location']['x']
/Users/doguma/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-45-97308dacfe21>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp2['latitude'][i] = locat[0]['locat

In [46]:
temp2

,tweet,separate camel cases,replace acronyms,remove stopwords,longitude,latitude
1,Cleared: Incident on #SouthernStateParkway WB ...,Cleared Incident on Southern State Parkway W...,cleared incident on southern state park way ...,cleared incident southern state park way west ...,-73.827778,40.792427
2,Updated: Incident on #RiverLine Both direction...,Updated Incident on River Line Both directio...,updated incident on river line both directio...,updated incident river line directions trenton...,0.000000,0.000000
3,Cleared: Incident on #NJ139Lower EB at New Jer...,Cleared Incident on NJ139Lower EB at New Jer...,cleared incident on new jersey 139lower ea...,cleared incident new jersey 139lower east boun...,-74.153169,40.703131
4,Construction on #I84E EB at Exit 39A (RTE 9 SB...,Construction on I84E EB at Exit 39A RTE 9 SB...,construction on interstate 84 e east bound ...,construction interstate 84 e east bound exit 3...,0.000000,0.000000
5,Incident on #NJ181 Both directions at CR 517/S...,Incident on NJ181 Both directions at CR 517 S...,incident on new jersey 181 both directions a...,incident new jersey 181 directions cr 517 spa ...,-74.007140,40.714550
6,Incident on #Q15ABus from Roosevelt Ave: Main ...,Incident on Q15ABus from Roosevelt Ave Main ...,incident on q15abus from roosevelt ave main ...,incident q15abus roosevelt ave main st rooseve...,-73.588960,40.678550
7,Cleared: Construction on #US202 NB from North ...,Cleared Construction on US202 NB from North ...,cleared construction on us202 no route h bo...,cleared construction us202 route h bound route...,-76.706877,39.891579
8,Incident on #Q12Bus from Roosevelt Ave: Main S...,Incident on Q12Bus from Roosevelt Ave Main S...,incident on q12bus from roosevelt ave main s...,incident q12bus roosevelt ave main st roosevel...,-73.588960,40.678550
9,Incident on #NJ3 EB at East of Garden State Pa...,Incident on NJ3 EB at East of Garden State Pa...,incident on new jersey 3 east bound at eas...,incident new jersey 3 east bound east garden s...,-105.717124,39.119262
10,Cleared: Incident on #ELine Manhattan bound at...,Cleared Incident on ELine Manhattan bound at...,cleared incident on eline manhattan bound at...,cleared incident eline manhattan bound 7th ave...,-74.020712,40.648715


In [48]:
temp2 = temp2[temp2['latitude'] !=0]
# temp2.drop(columns = ['separate camel cases', 'replace acronyms'], inplace=True)
temp2.tail(10)

,tweet,separate camel cases,replace acronyms,remove stopwords,longitude,latitude
12,Cleared: Construction on #NY30 Both directions...,Cleared Construction on NY30 Both directions...,cleared construction on new york 30 both di...,cleared construction new york 30 directions ne...,-1.045049,53.880315
13,Incident on #Q48Bus from Roosevelt Ave: Main S...,Incident on Q48Bus from Roosevelt Ave Main S...,incident on q48bus from roosevelt ave main s...,incident q48bus roosevelt ave main st roosevel...,-73.588960,40.678550
14,Incident on #US9 Both directions at County Rou...,Incident on US9 Both directions at County Rou...,incident on U.S. route 9 both directions at...,incident U S route 9 directions county route 3...,-73.917929,40.868738
16,Updated: Incident on #BM3 Both directions at 5...,Updated Incident on BM3 Both directions at 5...,updated incident on bm3 both directions at 5...,updated incident bm3 directions 57 st 2nd ave ...,-74.007140,40.714550
17,Updated: Incident on #M101Bus at Second Ave ht...,Updated Incident on M101Bus at Second Ave ht...,updated incident on m101bus at second ave ht...,updated incident m101bus second ave https co u...,-73.988656,40.727283
19,Cleared: Construction on #92Bus Both direction...,Cleared Construction on 92Bus Both direction...,cleared construction on 92bus both direction...,cleared construction 92bus directions scotland...,-73.976250,40.621120
20,Cleared: Construction on #Q32Bus at E 60th Str...,Cleared Construction on Q32Bus at E 60th Str...,cleared construction on q32bus at e 60th str...,cleared construction q32bus e 60th street lexi...,-73.970674,40.764026
21,Incident on #BM3 Both directions at 57 St and ...,Incident on BM3 Both directions at 57 St and ...,incident on bm3 both directions at 57 st and ...,incident bm3 directions 57 st 2nd ave https co...,-74.007140,40.714550
24,Incident on #Q15Bus from Roosevelt Ave: Main S...,Incident on Q15Bus from Roosevelt Ave Main S...,incident on q15bus from roosevelt ave main s...,incident q15bus roosevelt ave main st roosevel...,-73.588960,40.678550
25,Incident on #BrooklynBridge EB at Brooklyn Bou...,Incident on Brooklyn Bridge EB at Brooklyn Bo...,incident on brooklyn bridge east bound at...,incident brooklyn bridge east bound brooklyn b...,29.769650,-24.868210


In [8]:
temp2.columns = ['original tweet', 'neutral score', 'polar score', 'trimmed tweet', 'latitude', 'longitude']
temp2.to_csv('data_twitter/locat_6a.csv', mode='a')
temp2


In [37]:

temp2.to_csv('data_twitter/locat_6a.csv', mode='a')

### Count (frequency of location) check to find any unresolved acronyms or issues in Text processing
- if certain location is mentioned too many times - more than 100 times - it is likely  that geocoding for specific type of strings are inaccurate

In [38]:

freq = pd.DataFrame()
freq2 = temp2.groupby(["longitude", "latitude"]).size().reset_index(name="Frequency")

freq2 = freq2.sort_values('Frequency', ascending=False)

freq2.drop(freq2.head(1).index,inplace=True)
# most frequent value is the default value that returns location of 'new york'

freq2[['polar score', 'neutral score']] = temp2[['polar score', 'neutral score']]

freq2

,longitude,latitude,Frequency,polar score,neutral score
175,-73.990360,40.692450,84,0.0000,1.000
440,-73.829990,40.714000,46,0.0000,1.000
301,-73.946793,40.680002,44,0.0000,1.000
72,-74.080140,40.644550,39,0.1027,0.833
320,-73.942942,40.849800,33,0.0000,1.000
...,...,...,...,...,...
375,-73.911000,40.876410,1,0.1027,0.903
376,-73.909710,40.860190,1,0.1027,0.903
377,-73.909344,40.865739,1,0.1027,0.903
378,-73.906270,40.849390,1,0.0000,1.000
